In [25]:
#libraries
!pip install pyfastx
!pip3 install numpy 

#modules
import pyfastx
import numpy as np

In [27]:
from google.colab import files
uploaded = files.upload()

Saving GCF_000008865.2_ASM886v2_genomic.fa to GCF_000008865.2_ASM886v2_genomic (1).fa


In [28]:
for name, seq in pyfastx.Fasta('GCF_000008865.2_ASM886v2_genomic.fa', build_index = False):
  print (name,seq)

NC_002128.1 ...
NC_002127.1 TTCTTCTGCGAGTTCGTGCAGCTTCTCACACATGGTGGCCTGCTCGTCAGCATCGAGTGCGTCCAGTTTTTCGAGCAGCGTCAGGCTCTGGCTTTTTATGAATCCCGCCATGTTGAGTGCAGTTTGCTGCTGCTTGTTCATCTTTCTGTTTTCTCCGTTCTGTCTGTCATCTGCGTCGTGTGATTATATCGCGCACCACTTTTCGACCGTCTTACCGCCGGTATTCTGCCGACGGACATTTCAGTCAGACAACACTGTCACTGCCAAAAAACAGCAGTGCTTTGTTGGTAATTCGAACTTGCAGACAGGACAGGATGTGCAATTGTTATACCGCGCATACATGCACGCTATTACAATTACCCTGGTCAGGGCTTCGCCCCGACACCCCATGTCAGATACGGAGCCATGTTTTATGACAAAACGAAGTGGAAGTAATACGCGCAGGCGGGCTATCAGTCGCCCTGTTCGTCTGACGGCAGAAGAAGACCAGGAAATCAGAAAAAGGGCTGCTGAATGCGGCAAGACCGTTTCTGGTTTTTTACGGGCGGCAGCTCTCGGTAAGAAAGTTAACTCACTGACTGATGACCGGGTGCTGAAAGAAGTTATGCGACTGGGGGCGTTGCAGAAAAAACTCTTTATCGACGGCAAGCGTGTCGGGGACAGAGAGTATGCGGAGGTGCTGATCGCTATTACGGAGTATCACCGTGCCCTGTTATCCAGGCTTATGGCAGATTAGCTTCCCGGAGAGAAACTGTCGAAAACAGACGGTATGAACGCCGTAAGCCCCCAAACCGATCGCCATTCACTTTCATGCATAGCTATGCAGTGAGCTGAAAGCGATCCTGACGCATTTTTCCGGTTTACCCCGGGGAAAACATCTCTTTTTGCGGTGTCTGCGTCAGAATCGCGTTCAGCGCGTTTTGGCGGTGCGCGTAATGAGACGTTATGGTAAATGTCTTCTGGCTTGATATT

In [29]:
c = 1
for name, seq in pyfastx.Fasta('GCF_000008865.2_ASM886v2_genomic.fa', build_index = False):
  print (c, len(seq), seq) 
  c = c+1

1 92721 ...
2 3306 TTCTTCTGCGAGTTCGTGCAGCTTCTCACACATGGTGGCCTGCTCGTCAGCATCGAGTGCGTCCAGTTTTTCGAGCAGCGTCAGGCTCTGGCTTTTTATGAATCCCGCCATGTTGAGTGCAGTTTGCTGCTGCTTGTTCATCTTTCTGTTTTCTCCGTTCTGTCTGTCATCTGCGTCGTGTGATTATATCGCGCACCACTTTTCGACCGTCTTACCGCCGGTATTCTGCCGACGGACATTTCAGTCAGACAACACTGTCACTGCCAAAAAACAGCAGTGCTTTGTTGGTAATTCGAACTTGCAGACAGGACAGGATGTGCAATTGTTATACCGCGCATACATGCACGCTATTACAATTACCCTGGTCAGGGCTTCGCCCCGACACCCCATGTCAGATACGGAGCCATGTTTTATGACAAAACGAAGTGGAAGTAATACGCGCAGGCGGGCTATCAGTCGCCCTGTTCGTCTGACGGCAGAAGAAGACCAGGAAATCAGAAAAAGGGCTGCTGAATGCGGCAAGACCGTTTCTGGTTTTTTACGGGCGGCAGCTCTCGGTAAGAAAGTTAACTCACTGACTGATGACCGGGTGCTGAAAGAAGTTATGCGACTGGGGGCGTTGCAGAAAAAACTCTTTATCGACGGCAAGCGTGTCGGGGACAGAGAGTATGCGGAGGTGCTGATCGCTATTACGGAGTATCACCGTGCCCTGTTATCCAGGCTTATGGCAGATTAGCTTCCCGGAGAGAAACTGTCGAAAACAGACGGTATGAACGCCGTAAGCCCCCAAACCGATCGCCATTCACTTTCATGCATAGCTATGCAGTGAGCTGAAAGCGATCCTGACGCATTTTTCCGGTTTACCCCGGGGAAAACATCTCTTTTTGCGGTGTCTGCGTCAGAATCGCGTTCAGCGCGTTTTGGCGGTGCGCGTAATGAGACGTTATGGTAAATGTCTTCTGGCTTGATATTATATTGGAA

In [30]:
fa = pyfastx.Fasta('GCF_000008865.2_ASM886v2_genomic.fa')
print (type(fa), fa.size)
for i in range(len(fa)):
  print (type(fa[i]), len(fa[i]))

<class 'Fasta'> 96027
<class 'Sequence'> 92721
<class 'Sequence'> 3306


## extract Test sequence to encode embeddings

In [34]:
seq = fa[0]
start,stop = 0,800
k = 3
test_seq = seq[start:stop] #<------------------------------- INPUT TEST SEQUENCE, just part of the all DNA 
L = stop-start
n = L-k+1
print (len(test_seq))
print('slice of seq', test_seq[10:101])

# n_of_inputs = L -k +s
# L = tot lenght of sequence
# K = kmers size window
# s = sliding, 1 is standard


800
slice of seq TACCCGCCCATCCTAAAGAAGGGGATAGTCAACCACATCTGACCAGCCTGCGGAAAAGTCTGCTGCTTGTCCGTCCGGTGAAAGCTGATGA


## create a window of size k on the test_seq

In [0]:
def window(fseq, window_size):
    for i in range(len(fseq) - window_size + 1):
        yield fseq[i:i+window_size]

In [0]:
input = list()
for seq in window(test_seq, k):
    input.append(seq)
    # (seq)

### peeking into the k-mers

In [39]:
print (type(input))
c = 0
for i in input:
  c = c+1
  if c %100 == 0:
    c = c+1
    print (c,i)
    

<class 'list'>
101 GAT
201 CAT
301 GCC
401 GGA
501 AAC
601 CTG
701 CTG
801 ATT


In [40]:
print( "input is our test list of kmer = %2d sized words "%k)

input is our test list of kmer =  3 sized words 


# Input into CBOW

In [41]:
import torch
import torch.nn as nn
import numpy as np

CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
EMDEDDING_DIM = pow(4,k) 
print ('Embedding dimension',EMDEDDING_DIM)

word_to_ix = {}
ix_to_word = {}

vocab = input
print (type(vocab))
print ('all vocabulary, size :', len(vocab))

vocab_size = len(vocab)

for i, word in enumerate(vocab):
    word_to_ix[word] = i
    ix_to_word[i] = word

raw_text = input
data = []
c = 1
for i in range(2, len(raw_text) - 2): #first word to have 2 words before is the "third" one (0,1,2)
    context = (raw_text[i - 2], raw_text[i - 1],
               raw_text[i + 1], raw_text[i + 2])
    target = raw_text[i]
    c = c+1
    data.append((context, target))


Embedding dimension 64
<class 'list'>
all vocabulary, size : 798


# CBOW-model

In [0]:
def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)

def get_index_of_max(input):
    index = 0
    for i in range(1, len(input)):
        if input[i] > input[index]:
            index = i 
    return index

def get_max_prob_result(input, ix_to_word):
    return ix_to_word[get_index_of_max(input)]

In [0]:
class CBOW(torch.nn.Module):

    def __init__(self, vocab_size, embedding_dim):
        super(CBOW, self).__init__()

        #out: 1 x emdedding_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim) #used predefined nn.Embedding
        self.linear1 = nn.Linear(embedding_dim, 128)
        self.activation_function1 = nn.ReLU()
        
        #out: 1 x vocab_size
        self.linear2 = nn.Linear(128, vocab_size)
        self.activation_function2 = nn.LogSoftmax(dim = -1)
        

    def forward(self, inputs):
        embeds = sum(self.embeddings(inputs)).view(1,-1)
        out = self.linear1(embeds)
        out = self.activation_function1(out)
        out = self.linear2(out)
        out = self.activation_function2(out)
        return out

    def get_word_emdedding(self, word):
        word = torch.LongTensor([word_to_ix[word]])
        return self.embeddings(word).view(1,-1)


## Train the model
- for kmer =3 takes little time (vocabulary size = 64)
- for kmer = 7 takes much more (vocabulary size >16000)

In [0]:
model = CBOW(vocab_size, EMDEDDING_DIM)

loss_function = nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

for epoch in range(50):
    total_loss = 0
    for context, target in data:
        context_vector = make_context_vector(context, word_to_ix)  
        model.zero_grad()
        log_probs = model(context_vector)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))
        loss.backward()
        optimizer.step()

        total_loss += loss.data

## Test Run 

In [46]:
## ====================== TEST
i = 4
context_test = (input[i - 2], input[i - 1],input[i + 1], input[i + 2])
target_test = input[i]
#print ('target_test',target_test)
#print (context_test, target_test)

print ('context')
for i in context_test:
  print (i)
context_vector = make_context_vector(context_test, word_to_ix)
a = model(context_vector).data.numpy()
prediction = get_max_prob_result(a[0], ix_to_word)
print ()
print ('model prediction:', prediction)
print ('target_test', target_test)

context
CCA
CAG
GAT
ATT

model prediction: AGA
target_test AGA


## Get the embeddings for each word size k-mer ub the test_seq

All word embeddings are tensors collected into a matrix.

In [47]:
word = input[4]
c= 0
dash = '-'*40
tensor_matrix = list()
for word in input: 
  c= c+1
  e = model.get_word_emdedding(word)
  tensor_matrix.append(e)
  if c % 100 ==0:
    print(dash)
    print('word tensor number %3d aka string %8s'%(c,word))
    print (e)

#model.get_word_emdedding(word)

----------------------------------------
word tensor number 100 aka string      GAT
tensor([[ 0.8425, -1.4560, -0.9374, -0.0591, -0.5922, -0.0520, -1.3109, -0.0565,
         -1.2968, -0.5374, -2.5477,  1.0762, -0.0676, -0.6041, -0.9613, -0.8833,
          1.2449, -0.5095,  1.7819,  0.5408,  0.1450, -1.4858, -1.1683, -0.4233,
          0.9256, -0.0633, -2.2710,  0.2143,  0.3967,  0.6100, -0.4722, -0.0167,
          0.5080, -0.9862, -0.1028,  0.1463,  1.0610,  2.3068,  0.4224, -0.6590,
          1.3854, -0.5594, -1.1197,  0.0268, -0.7796,  0.6338, -0.0674, -0.9681,
          1.2538, -0.0971,  0.0642,  0.7553, -0.1334,  0.1145, -0.7343, -1.5783,
         -1.6578,  0.2195,  0.0459, -1.4285,  0.6836, -1.9512, -0.4442, -0.6714]],
       grad_fn=<ViewBackward>)
----------------------------------------
word tensor number 200 aka string      ATC
tensor([[ 0.5699, -0.8164, -0.1501,  0.1422,  0.4819, -0.3259, -0.0085, -1.1018,
         -1.3494,  0.6041, -1.7978,  0.4962, -1.9202, -0.4118,  0.4757